In [1]:
import os
import glob
from scipy.io import wavfile
import scipy.io.wavfile as wav
import librosa
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
import random


from python_speech_features import mfcc
from python_speech_features import logfbank


from tensorflow.keras.models import Sequential

In [56]:
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# import keras
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.utils import to_categorical,np_utils
# from keras.layers import Dense, Input, GlobalMaxPooling1D,Dropout, Activation, Flatten
# from keras.layers import Conv1D, MaxPooling1D, Embedding, Convolution2D, MaxPooling2D,Conv2D, Reshape, GRU, TimeDistributed, Lambda
# from keras.models import Model

# from keras.preprocessing import sequence
# from keras.models import Sequential
# from keras.layers import Dense, Embedding
# from keras.layers import LSTM , Bidirectional,Dropout, BatchNormalization
# from keras import backend as K
# from keras.layers.advanced_activations import LeakyReLU
# from keras import regularizers
# from keras.layers.merge import add, concatenate
# from keras.callbacks import ModelCheckpoint

In [63]:
path = r"data\recordings"
files = os.listdir(path)
audio_data=[]
for filename in glob.glob(os.path.join(path, '*.wav')):
    samplerate, data = wavfile.read(filename)
    audio_data.append(data)

C:\Users\maruf\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: WavFileWarning: Chunk (non-data) not understood, skipping it.
  """


In [3]:
label=[]
for i in files:
    label.append(i[0])

In [4]:
raw_aud_data=[]
for filename in glob.glob(os.path.join(path, '*.wav')):
    (rate,sig) = wav.read(filename)
    raw_aud_data.append(sig)

C:\Users\maruf\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
final_audio_trans=label

In [6]:
raw_aud_data[10].shape

(5235,)

In [7]:
raw_aud_data_flat=[]
for i in raw_aud_data:
    raw_aud_data_flat.append(i.reshape(1,-1))

In [8]:
raw_aud_data_flat[100].shape

(1, 3142)

In [9]:
final_audio_trans[0]

'0'

In [10]:
label_int=[]
for i in label:
    label_int.append(int(i))
    

In [11]:
import inflect

p = inflect. engine()
final_audio_trans=[]
for i in label_int:
    final_audio_trans.append(p. number_to_words(i))


In [12]:
samp_id=list(range(0,1500)) #1500 is the number of data


In [13]:
#shuffling multiple times
for _ in range(3):
    random.shuffle(samp_id)


In [14]:
#Saving sample_id as text
with open("samp_id.txt", "wb") as fp:
    pickle.dump(samp_id, fp)
    

In [15]:
#Loading sample_id as text
with open("samp_id.txt", "rb") as fp:
    samp_id = pickle.load(fp)    


In [16]:
#shuffled training data
raw_aud_data_flat_shuffle=[]
final_audio_trans_shuffled=[]
final_audio_trans_shuffled=[]
for i in samp_id:
    raw_aud_data_flat_shuffle.append(raw_aud_data_flat[i])
    final_audio_trans_shuffled.append(final_audio_trans[i])

In [17]:
final_aud_list=final_audio_trans_shuffled

In [18]:
target_texts = []                            
output_vocabulary = set()
start_token = '@'                                             
stop_token = '#'

for i in final_aud_list:
    target_text = start_token + i + stop_token
    target_texts.append(target_text)
    
    for char in target_text:
        if char not in output_vocabulary:
            output_vocabulary.add(char)    

In [19]:
#function to get the actual word-characters till # appears in the text
def actual_word(mytextlist):
    revised_text=[]
    for i in mytextlist:
        ct=[]
        if "#" in i:
            for j in range(i.index("#")):
                ct.append(i[j])
            revised_text.append("".join(ct))
        else:
            revised_text.append(i)
            
        #revised_text.append("".join(ct))
    return revised_text   

In [20]:

max_decoder_seq_length = max([len(txt) for txt in target_texts])
target_token_index = dict([(char, i) for i, char in enumerate(output_vocabulary)])
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
output_vocab_size=len(output_vocabulary)

In [21]:
reverse_target_char_index

{0: 'r',
 1: 'o',
 2: 'w',
 3: '@',
 4: 'u',
 5: '#',
 6: 'f',
 7: 'x',
 8: 'z',
 9: 'h',
 10: 'v',
 11: 't',
 12: 'e',
 13: 'n',
 14: 's',
 15: 'i',
 16: 'g'}

In [22]:
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, output_vocab_size), dtype='float32')
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, output_vocab_size), dtype='float32')

In [23]:
#encoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, output_vocab_size), dtype='float32')
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, output_vocab_size), dtype='float32')
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, output_vocab_size), dtype='float32')

In [24]:
for i, text in enumerate(target_texts):  
    for t, char in enumerate(text):
            decoder_input_data[i, t, target_token_index[char]] = 1.
            if t > 0:
                decoder_target_data[i, t - 1, target_token_index[char]] = 1

In [25]:
#This is for input text
#for i, text in enumerate(input_text):  
#    for t, char in enumerate(text):
#            decoder_input_data[i, t, target_token_index[char]] = 1.
#            if t > 0:
#                decoder_target_data[i, t - 1, target_token_index[char]] = 1

In [158]:
#Saving decoder_input_data as text

with open("mfcc_data_shuffled.txt", "wb") as fp:
    pickle.dump(mfcc_data_shuffled, fp)

with open("decoder_input_data.txt", "wb") as fp:
    pickle.dump(decoder_input_data, fp)
    
#Saving decoder_output_data as text
with open("decoder_target_data.txt", "wb") as fp:
    pickle.dump(decoder_target_data, fp)
    
#Saving target_token_index as text
with open("target_token_index.txt", "wb") as fp:
    pickle.dump(target_token_index, fp)
    
#Saving reverse_target_char_index as text
with open("reverse_target_char_index.txt", "wb") as fp:
    pickle.dump(reverse_target_char_index, fp)
    
    
    
    
    

In [27]:
#Loading decoder_input_data as text
with open("mfcc_data_shuffled.txt", "rb") as fp:
    mfcc_data_shuffled = pickle.load(fp)    


#Loading decoder_input_data as text
with open("decoder_input_data.txt", "rb") as fp:
    decoder_input_data = pickle.load(fp)    

#Loading decoder_output_data as text
with open("decoder_target_data.txt", "rb") as fp:
    decoder_target_data = pickle.load(fp)
    
#Loading decoder_output_data as text
with open("target_token_index.txt", "rb") as fp:
    target_token_index = pickle.load(fp)
    
#Loading decoder_output_data as text
with open("reverse_target_char_index.txt", "rb") as fp:
    reverse_target_char_index = pickle.load(fp)

In [32]:
#Transposing 
mfcc_data_shuffled_transposed=[]
for i in range(len(mfcc_data_shuffled)):
    mfcc_data_shuffled_transposed.append(np.transpose(mfcc_data_shuffled[i]))

NameError: name 'mfcc_data_shuffled' is not defined

In [26]:
tc=[]
for i in raw_aud_data_flat:
    tc.append(i.shape)

In [28]:
all_data=[]
for i in range(len(raw_aud_data_flat_shuffle)):
    all_data.append(raw_aud_data_flat_shuffle[i].tolist()[0])
#    for j in range(len(a)):
#        all_data.append(a[j])

all_data_shuffled_padded = tf.keras.preprocessing.sequence.pad_sequences(all_data,
                                                         padding='post')        

In [29]:
raw_aud_data_flat_shuffle_padded=[]
for i in all_data_shuffled_padded:
    raw_aud_data_flat_shuffle_padded.append(i.reshape(i.shape[0],1))

In [30]:
raw_aud_data_flat_shuffle_padded[0].shape    

(18262, 1)

In [31]:
mfcc_data_shuffled_padded=[]
for i in range(1500):
    inter=all_data_shuffled_padded[13*i:13*(i+1)]
    mfcc_data_shuffled_padded.append(np.array(inter))


In [32]:
all_data_shuffled_padded[0]

array([126, 126, 124, ...,   0,   0,   0])

In [33]:
#Transposing 
mfcc_data_shuffled_padded_transposed=[]
for i in range(len(mfcc_data_shuffled_padded)):
    mfcc_data_shuffled_padded_transposed.append(np.transpose(mfcc_data_shuffled_padded[i]))

In [34]:
mfcc_data_shuffled_padded_transposed[0].shape


(18262, 13)

In [35]:
decoder_input_data.shape

(1500, 7, 17)

In [36]:
max_decoder_seq_length

7

In [51]:
encoder_inputs = tf.keras.Input(shape=(18262,1))
conv1= Conv1D(filters=32, kernel_size = 4, padding='causal',activation='relu')(encoder_inputs)
conv2= Conv1D(filters=64, kernel_size = 4, strides=2,padding='causal',activation='relu')(conv1)
conv3= Conv1D(filters=128, kernel_size = 4, strides=2,padding='causal',activation='relu')(conv2)
conv4= Conv1D(filters=256, kernel_size = 4, strides=2,padding='causal',activation='relu')(conv3)
conv5= Conv1D(filters=512, kernel_size = 4, strides=3,padding='causal',activation='relu')(conv4)
conv6= Conv1D(filters=1024, kernel_size = 5, strides=4,padding='causal',activation='relu')(conv5)
conv7= Conv1D(filters=1024, kernel_size = 5, strides=5,padding='causal',activation='relu')(conv6)


encoder = LSTM(256, return_state=True, return_sequences=True)
encoder_outputs, state_h, state_c = encoder(conv7)
encoder_states = [state_h, state_c]
 
decoder_inputs = Input(shape=(max_decoder_seq_length, output_vocab_size))
decoder_lstm = LSTM(256, return_sequences=True,
                     return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
     initial_state=encoder_states)
decoder_dense = Dense(output_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
 


In [52]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 18262, 1)]   0                                            
__________________________________________________________________________________________________
conv1d_21 (Conv1D)              (None, 18262, 32)    160         input_9[0][0]                    
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, 9131, 64)     8256        conv1d_21[0][0]                  
__________________________________________________________________________________________________
conv1d_23 (Conv1D)              (None, 4566, 128)    32896       conv1d_22[0][0]                  
______________________________________________________________________________________________

In [37]:
def correctness(y_true, y_pred):
    vec_oh=[]
    #a=y_pred

    #vec_oh=[]
    #a=train_y.copy()
    if type(y_pred.shape[0]) != None:
        for i in range(y_pred.shape[0]):
            ppp=[]
            for j in range(y_pred.shape[1]):

                if y_pred[i][j].max(axis=0)!=0:
                    ppp.append((y_pred[i][j] == y_pred[i][j].max(axis=0)).astype(int))
                else:
                    
                    ppp.append((y_pred[i][j]).astype(int))
            b=np.array(ppp)
            vec_oh.append(b)
            vec_oh_array=np.array(vec_oh)
        
        check=np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        for i in range(vec_oh_array.shape[0]):
            ind=[]
            for j in range(vec_oh_array.shape[1]):
                if np.array_equal(vec_oh_array[i][j],check)==True:
                    ind.append(j)
                    ind=sorted(ind)
            for k in ind[1:]:
                vec_oh_array[i][k][0]=0
              
        
        
        sentence_accuracy=[]
        for i in range(vec_oh_array.shape[0]):
                sentence_accuracy.append(np.array_equal(vec_oh_array[i],train_y[i]))
        no_correct_sentence=sum(sentence_accuracy)
        percn_correct_sentence=sum(sentence_accuracy)/len(sentence_accuracy)

        character_accuracy=[]
        for i in range(vec_oh_array.shape[0]):
            for j in range(vec_oh_array.shape[1]):
                character_accuracy.append(np.array_equal(vec_oh_array[i][j],train_y[i][j]))
        no_correct_character=sum(character_accuracy)
        percn_correct_character=sum(character_accuracy)/len(character_accuracy)
        
        return no_correct_sentence, percn_correct_sentence, no_correct_character,percn_correct_character
        #return vec_oh_array
    

In [53]:
from tensorflow.keras.optimizers import Adam

Optimizer = Adam(lr = .001)

In [54]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=["accuracy"])

In [57]:
# filepath_best = "weights\weights.{epoch:02d}-{loss:.2f}-bigger_CNN-Raw.hdf5" #this will save the model with best weights
# checkpoint_best = ModelCheckpoint(filepath_best, monitor='loss', verbose=1,
#     save_best_only=True, mode='auto', period=1)

# filepath_latest = "weights\latest_weights_CNN-Raw.hdf5" #this will save the model with latest weight updates (not necessarily the best weights)
# checkpoint_latest = ModelCheckpoint(filepath_latest, monitor='loss', verbose=1,
#     save_best_only=False, mode='auto', period=1)

In [58]:
#Splitting into train and test sets-5% data as test set
#Test sets
test_size=len(raw_aud_data_flat_shuffle_padded)*.05

raw_aud_data_flat_shuffle_padded_test=raw_aud_data_flat_shuffle_padded[0:round(test_size)]
decoder_input_data_test=decoder_input_data[0:round(test_size)]
decoder_target_data_test=decoder_target_data[0:round(test_size)]
target_texts_test=target_texts[0:round(test_size)]

#Training sets
raw_aud_data_flat_shuffle_padded_train=raw_aud_data_flat_shuffle_padded[round(test_size):]
decoder_input_data_train=decoder_input_data[round(test_size):]
decoder_target_data_train=decoder_target_data[round(test_size):]
target_texts_train=target_texts[round(test_size):]


In [230]:
#Saving decoder_input_data as text

with open("raw_aud_data_flat_shuffle_padded_test.txt", "wb") as fp:
    pickle.dump(raw_aud_data_flat_shuffle_padded_test, fp)

with open("decoder_input_data_test.txt", "wb") as fp:
    pickle.dump(decoder_input_data_test, fp)
    
#Saving decoder_output_data as text
with open("decoder_target_data_test.txt", "wb") as fp:
    pickle.dump(decoder_target_data_test, fp)
    
#Saving target_token_index as text
with open("target_texts_test.txt", "wb") as fp:
    pickle.dump(target_texts_test, fp)
    
#Saving reverse_target_char_index as text
with open("raw_aud_data_flat_shuffle_padded_train.txt", "wb") as fp:
    pickle.dump(raw_aud_data_flat_shuffle_padded_train, fp)
    
with open("decoder_input_data_train.txt", "wb") as fp:
    pickle.dump(decoder_input_data_train, fp)
    
with open("decoder_target_data_train.txt", "wb") as fp:
    pickle.dump(decoder_target_data_train, fp)
    
with open("target_texts_train.txt", "wb") as fp:
    pickle.dump(target_texts_train, fp)
    
    

In [41]:
#Loading raw_aud_data_flat_shuffle_padded_test as text
with open("raw_aud_data_flat_shuffle_padded_test.txt", "rb") as fp:
    raw_aud_data_flat_shuffle_padded_test = pickle.load(fp)    


#Loading decoder_input_data_test as text
with open("decoder_input_data_test.txt", "rb") as fp:
    decoder_input_data_test = pickle.load(fp)    

#Loading decoder_target_data_test as text
with open("decoder_target_data_test.txt", "rb") as fp:
    decoder_target_data_test = pickle.load(fp)
    
#Loading target_texts_test as text
with open("target_texts_test.txt", "rb") as fp:
    target_texts_test = pickle.load(fp)
    
#Loading raw_aud_data_flat_shuffle_padded_train as text
with open("raw_aud_data_flat_shuffle_padded_train.txt", "rb") as fp:
    raw_aud_data_flat_shuffle_padded_train = pickle.load(fp)
    
#Loading decoder_input_data_train as text
with open("decoder_input_data_train.txt", "rb") as fp:
    decoder_input_data_train = pickle.load(fp)

#Loading decoder_target_data_train as text
with open("decoder_target_data_train.txt", "rb") as fp:
    decoder_target_data_train = pickle.load(fp)
    
#Loading target_texts_train as text
with open("target_texts_train.txt", "rb") as fp:
    target_texts_train = pickle.load(fp)

In [59]:
#batch function
def batch_custom(xdataset,size):
    id=[]
    for m in range(0,len(xdataset),size):
        id.append(m)

    sl=[]
    for m in range(0,len(xdataset)):
                   sl.append(m)
    mybatch=[]
    for k in range(len(id)-1):
        mybatch.append(sl[id[k]:id[k+1]])
        
    return mybatch

In [60]:
#batch of 32
batch_ids=batch_custom(raw_aud_data_flat_shuffle_padded_train,32)

In [66]:
for i in range(1000):
    for k in batch_ids:            
        batch_input=[raw_aud_data_flat_shuffle_padded_train[i] for i in k]
        batch_input = np.array(batch_input)
        batch_input = batch_input/np.max(batch_input)
        decoder_input_batch=decoder_input_data_train[k[0]:k[-1]+1]        
        decoder_output_batch=decoder_target_data_train[k[0]:k[-1]+1]        
#         history=model.fit([batch_input, decoder_input_batch], decoder_output_batch, batch_size=32,epochs=1, callbacks=[checkpoint_best,checkpoint_latest])
        history=model.fit([batch_input, decoder_input_batch], decoder_output_batch, batch_size=32,epochs=1)


Train on 32 samples
32/32 [==============================] - 0s 10ms/sample - loss: 2.0518 - accuracy: 0.2143
Train on 32 samples
32/32 [==============================] - 0s 8ms/sample - loss: 1.8743 - accuracy: 0.1429
Train on 32 samples
32/32 [==============================] - 0s 8ms/sample - loss: 1.7868 - accuracy: 0.1518
Train on 32 samples
32/32 [==============================] - 0s 8ms/sample - loss: 1.7339 - accuracy: 0.1830
Train on 32 samples
32/32 [==============================] - 0s 8ms/sample - loss: 1.7818 - accuracy: 0.1562
Train on 32 samples
32/32 [==============================] - 0s 8ms/sample - loss: 1.5993 - accuracy: 0.2232
Train on 32 samples
32/32 [==============================] - 0s 9ms/sample - loss: 1.6449 - accuracy: 0.1875
Train on 32 samples
32/32 [==============================] - 0s 9ms/sample - loss: 1.6462 - accuracy: 0.2054
Train on 32 samples
32/32 [==============================] - 0s 8ms/sample - loss: 1.5425 - accuracy: 0.1920
Train on 32 sample

KeyboardInterrupt: 

In [1]:
# from tensorflow.keras.models import Sequential, load_model 

# model = load_model("weights\latest_weights_CNN-Raw.hdf5",compile=False) 
                   

Using TensorFlow backend.


In [ ]:
#######################################Prediction experiment#######################################

In [32]:
thought_input = [Input(shape=(256,)),
     Input(shape=(256,))]                       
decoder_outputs, state_h, state_c = decoder_lstm(
     decoder_inputs, initial_state=thought_input)       
decoder_states = [state_h, state_c]                    
decoder_outputs = decoder_dense(decoder_outputs)       
 
decoder_model = Model(                                 
     inputs=[decoder_inputs] + thought_input,           
     output=[decoder_outputs] + decoder_states)         

C:\Users\maruf\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  # Remove the CWD from sys.path while we load stuff.


In [33]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [218]:
thought_train = encoder_model.predict(train_x)   
thought_test = encoder_model.predict(test_x)             


In [219]:
output_token_train, h, c = decoder_model.predict(
         [train_y] + thought_train) 
output_token_test, h, c = decoder_model.predict(
         [test_y] + thought_test) 

In [232]:
#Saving output_token_train as text
with open("thought_train_cnn.txt", "wb") as fp:
    pickle.dump(thought_train, fp)

with open("thought_test_cnn.txt", "wb") as fp:
    pickle.dump(thought_test, fp)
    

In [229]:
#Saving output_token_train as text
with open("output_token_train_cnn.txt", "wb") as fp:
    pickle.dump(output_token_train, fp)

with open("output_token_test_cnn.txt", "wb") as fp:
    pickle.dump(output_token_test, fp)
    

In [6]:
#Loading decoder_output_data as text
with open("thought_train_cnn.txt", "rb") as fp:
    thought_train = pickle.load(fp)

#Loading decoder_output_data as text
with open("thought_test_cnn.txt", "rb") as fp:
    thought_test = pickle.load(fp)

In [7]:
#Loading decoder_output_data as text
with open("output_token_test_cnn.txt", "rb") as fp:
    output_token_test = pickle.load(fp)

#Loading decoder_output_data as text
with open("output_token_train_cnn.txt", "rb") as fp:
    output_token_train = pickle.load(fp)

In [43]:
train_x=np.array(raw_aud_data_flat_shuffle_padded_train)
train_x.shape

(1425, 18262, 1)

In [44]:
test_x=np.array(raw_aud_data_flat_shuffle_padded_test)
test_x.shape

(75, 18262, 1)

In [45]:
train_y=decoder_target_data_train
train_y.shape

(1425, 7, 17)

In [46]:
test_y=decoder_target_data_test
test_y.shape

(75, 7, 17)

In [35]:
def sentence_prediction(output_tokens, actual_label_vector):
    
    predicted_sentence=[]
    actual_sentence=[]
    for i in range(output_tokens.shape[0]):
        generated_token_idx = np.argmax(output_tokens[i], axis=1)
        g=generated_token_idx.tolist()
        char=[]
        for i in g:
            char.append(reverse_target_char_index[i])
        predicted_sentence.append("".join(char))
    
    for i in range(actual_label_vector.shape[0]):
        generated_token_idx_actual=np.argmax(actual_label_vector[i], axis=1)
        ga=generated_token_idx_actual.tolist()
        char_label=[]
        for i in ga:
            char_label.append(reverse_target_char_index[i])
        actual_sentence.append("".join(char_label))
        #char_label.append(reverse_target_char_index[i])
        #actual_sentence.append("".join(char_label))
    return actual_sentence,predicted_sentence

        
    

In [47]:
iteration_train=correctness(train_y, output_token_train)
iteration_test=correctness(test_y, output_token_test)

In [48]:
#words with tokens
match_pred_act_train=sentence_prediction(output_token_train,train_y)
match_pred_act_test=sentence_prediction(output_token_test,test_y)

In [51]:
match_pred_act_train[1]

['sx#####',
 'sght###',
 'sne####',
 'sree###',
 'onr####',
 'sne####',
 'sgh####',
 'sne####',
 'sree###',
 'sven#n#',
 'sx#####',
 'ove####',
 'sien#n#',
 'sree###',
 'sght###',
 'so#####',
 'sve####',
 'sne####',
 'sne####',
 'so#####',
 'snr####',
 'nine###',
 'oven#n#',
 'sght###',
 'sgh####',
 'sve####',
 'sne####',
 'ove####',
 'sve####',
 'sien#n#',
 'sght###',
 'so#####',
 'tro####',
 'onr####',
 'sree###',
 'so#####',
 'sgh####',
 'nine###',
 'tro####',
 'snr####',
 'tro####',
 'sven#n#',
 'tro####',
 'sx#####',
 'sgh####',
 'tro####',
 'sgh####',
 'ni#e###',
 'sne####',
 'tro####',
 'tro####',
 'ni#e###',
 'sven#n#',
 'sx#####',
 'nine###',
 'nine###',
 'tro####',
 'sven#n#',
 'sght###',
 'sne####',
 'tro####',
 'snr####',
 'sne####',
 'onr####',
 'so#####',
 'oee####',
 'ni#e###',
 'onr####',
 'sgh####',
 'oven#n#',
 'sx#####',
 'sx#####',
 'sien#n#',
 'ni#e###',
 'ni#e###',
 'sree###',
 'sght###',
 'sgh####',
 'sght###',
 'sve####',
 'sven#n#',
 'onr####',
 'sven#n#',
 'ov

In [52]:
iteration_train

(0, 0.0, 3121, 0.3128822055137845)

In [53]:
iteration_test

(0, 0.0, 167, 0.3180952380952381)

In [65]:
#words without tokens
actualword_train_word=actual_word(match_pred_act_train[0])
predictedword__train_word=actual_word(match_pred_act_train[1])

actualword_test_word=actual_word(match_pred_act_test[0])
predictedword__test_word=actual_word(match_pred_act_test[1])

In [77]:
#Saving actualword_train_word as text
with open("actualword_train_word.txt", "wb") as fp:
    pickle.dump(actualword_train_word, fp)

with open("predictedword__train_word.txt", "wb") as fp:
    pickle.dump(predictedword__train_word, fp)

with open("actualword_test_word.txt", "wb") as fp:
    pickle.dump(actualword_test_word, fp)

with open("predictedword__test_word.txt", "wb") as fp:
    pickle.dump(predictedword__test_word, fp)


In [ ]:
#Saving actualword_train_word as text
with open("actualword_train_word.txt", "rb") as fp:
    actualword_train_word = pickle.load(fp)
with open("predictedword__train_word.txt", "rb") as fp:
    predictedword__train_word = pickle.load(fp)
with open("actualword_test_word.txt", "rb") as fp:
    actualword_test_word = pickle.load(fp)
with open("predictedword__test_word.txt", "rb") as fp:
    predictedword__test_word = pickle.load(fp)

In [82]:
train_word_comparison = pd.DataFrame((actualword_train_word,predictedword__train_word), index=["actual","predicted"]).T
test_word_comparison = pd.DataFrame((actualword_test_word,predictedword__test_word), index=["actual","predicted"]).T

In [83]:
test_word_comparison

,actual,predicted
0,three,sree
1,nine,sne
2,two,so
3,five,ove
4,three,sree
...,...,...
70,seven,sven
71,one,nine
72,six,sx
73,one,ni


In [85]:
train_word_comparison.to_csv('train_word_comparison.csv')
test_word_comparison.to_csv('test_word_comparison.csv')

In [55]:
def model_accuracy_wordlevel(predword,actword):
    val=[]
    for i in range(len(actword)):
        if actword[i]==predword[i]:
            val.append(1)
        else:
            val.append(0)
    return sum(val)

In [56]:
def model_accuracy_charevel(predword,actword):
    val=[]
    for i in range(len(actword)):
        for j in range(min(len(actword[i]),len(predword[i]))):
            if actword[i][j]==predword[i][j]:
                val.append(1)
            else:
                val.append(0)
    return sum(val)

In [61]:
model_accuracy_wordlevel(predictedword__train_word,actualword_train_word)

0

In [66]:
model_accuracy_wordlevel(predictedword__test_word,actualword_test_word)

0

In [67]:
model_accuracy_charevel(predictedword__train_word,actualword_train_word)

343

In [69]:
model_accuracy_charevel(predictedword__test_word,actualword_test_word)

22

In [71]:
def character_counts(train_words):
    total_characters=[]
    for i in range(len(train_words)):
        total_characters.append(len(train_words[i]))
    return sum(total_characters)

In [72]:
character_counts(actualword_train_word)

5702

In [73]:
character_counts(actualword_test_word)

298

In [76]:
343/5702

0.060154331813398805

In [62]:
#words without tokens
actualword_test_word=actual_word(match_pred_act_test[0])
predictedword__test_word=actual_word(match_pred_act_test[1])